## Duração da viagem de táxi em Nova York (Regressão)

Construir um modelo que prevê a duração total da viagem de táxi em Nova York. O primeiro conjunto de dados foi divulgado pela Comissão de Táxis e Limusines de Nova York, que inclui o tempo de coleta, as coordenadas geográficas, o número de passageiros e várias outras variáveis. Onde as features dos dataset são:

     - id: um identificador único para cada viagem
     - vendor_id: um código indicando o provedor associado ao registro de viagem
     - pickup_datetime: data e hora em que o medidor foi ativado
     - dropoff_datetime: data e hora em que o medidor foi desativado
     - passenger_count: o número de passageiros no veículo (valor inserido pelo motorista)
     - pickup_longitude: a longitude em que o medidor foi engajado
     - pickup_latitude: a latitude em que o medidor foi contratado
     - dropoff_longitude: a longitude em que o medidor foi desativado
     - dropoff_latitude: a latitude em que o medidor foi desativado
     - store_and_fwd_fla: Este indicador indica se o registro da viagem foi mantido na memória do veículo antes de ser enviado  ao fornecedor porque o veículo não tinha uma conexão com o servidor - Y = armazenar e encaminhar; N = não é uma loja e uma viagem para a frente
     - trip_duration: duração da viagem em segundos

### Importações

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns
from datetime import datetime
from datetime import date
from sklearn.metrics import mean_squared_log_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb


%matplotlib inline

#### Carregando os dados de traino

In [ ]:
df = pd.read_csv('../input/train.csv')

In [ ]:
test = pd.read_csv('../input/test.csv')

#### Analisando os dados

In [ ]:
df.head()

In [ ]:
print(df.shape)

In [ ]:
df.describe()

In [ ]:
df.info()

#### Observamos que as informações contidas em id, vendor_id e store_and_fwd_flag não influenciam no resultado da predição. Então vamos retirá-las de nossos dados.

In [ ]:
df_ = df.drop(['id','vendor_id','store_and_fwd_flag'],axis=1)

## É preciso salvar o Id para o teste para submeter a solução

In [ ]:
id_test = test['id']
test = test.drop(['id','vendor_id','store_and_fwd_flag'],axis=1)

In [ ]:
df_.head()

### Inclusão das features Hora, Data, Dia, Mês, Dia da Semana e turno

In [ ]:
df_['Hora'] = ([y[0:2] for x,y in df_['pickup_datetime'].str.split('\s').values])
test['Hora'] = ([y[0:2] for x,y in test['pickup_datetime'].str.split('\s').values])

In [ ]:
df_['Data'] = ([datetime.strptime(x, '%Y-%m-%d').date() for x,y in df_['pickup_datetime'].str.split('\s').values])
test['Data'] = ([datetime.strptime(x, '%Y-%m-%d').date() for x,y in test['pickup_datetime'].str.split('\s').values])

In [ ]:
df_['Dia'] = ([x.day for x in df_['Data']])
test['Dia'] = ([x.day for x in test['Data']])

In [ ]:
df_['Mês'] = ([x.month for x in df_['Data']])
test['Mês'] = ([x.month for x in test['Data']])

In [ ]:
df_['Mês'].unique()

In [ ]:
dias = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-Feira', 'Sexta-feira', 'Sábado', 'Domingo']
#df_['Dia_Semana'] = ([dias[x.weekday()] for x in df_['Data']])
df_['Dia_Semana'] = ([x.weekday() for x in df_['Data']])
test['Dia_Semana'] = ([x.weekday() for x in test['Data']])

### Tranformando a feature store_and_fwd_flag em numerica

In [ ]:
#df_['store_and_fwd_flag'] = np.where(df_['store_and_fwd_flag'] == 'N',0 , 1)
#test['store_and_fwd_flag'] = np.where(test['store_and_fwd_flag'] == 'N',0 , 1)

### Função para retornar o turno da viagem

In [ ]:
def turno(x):
    Turno = ['Manhã','Almoço','Tarde-Noite','Noite','Madrugada']
    if(7 <= x <= 11):
        return 0
    elif(12 <= x <= 14):
        return 1
    elif(15 <= x <= 19):
        return 2
    elif(20 <= x <= 23):
        return 3
    elif(0 <= x <= 6):
        return 4

In [ ]:
df_['Turno'] = ([turno(int(x)) for x in df_['Hora']])
test['Turno'] = ([turno(int(x)) for x in test['Hora']])

In [ ]:
df_.head()

In [ ]:
df_.shape, test.shape

In [ ]:
df_['passenger_count'].unique()

### Gráficos mostrando a frequencia dos nas features 'passenger_count','Turno','Dia','Mês','Dia_Semana'

In [ ]:
time = df_.set_index('trip_duration')

cols = ['passenger_count','Turno','Dia','Mês','Dia_Semana']

for c in cols:
    plt.figure()
    plt.title(c)
    time[c].plot(kind='hist')
    plt.show()

### Excluindo as features 'dropoff_datetime', 'pickup_datetime','Hora','Data' , pois não serão mais utilizadas

In [ ]:
df_.drop(['dropoff_datetime', 'pickup_datetime','Data'],axis=1, inplace=True)
test.drop(['pickup_datetime','Data'],axis=1, inplace=True)

In [ ]:
df_.head()

### Calculando a distancia das viagens com a distancia haversine

In [ ]:
def distancia(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon):
    
    #Definindo o raio da Terra (km)
    R_terra = 6371
    #Convertendo graus para radianos
    inicio_lat, inicio_lon, fim_lat, fim_lon = map(np.radians,[pickup_lat, pickup_lon, dropoff_lat, dropoff_lon])
    #Calculando as distancias de lat e long 
    dlat = fim_lat - inicio_lat
    dlon = fim_lon - inicio_lon
    
    #Calculando distancia haversine
    d = np.sin(dlat/2.0)**2 + np.cos(inicio_lat) * np.cos(fim_lat) * np.sin(dlon/2.0)**2
    
    return 2 * R_terra * np.arcsin(np.sqrt(d))

### Incluindo features de distancia da viagem

In [ ]:
df_['Distancia'] = distancia(df_['pickup_latitude'], df_['pickup_longitude'], 
                                   df_['dropoff_latitude'] , df_['dropoff_longitude'])
test['Distancia'] = distancia(test['pickup_latitude'], test['pickup_longitude'], 
                                   test['dropoff_latitude'] , test['dropoff_longitude'])

### Trocando a posição da label para a ultima coluna

In [ ]:
df_['Duração'] = df_['trip_duration']
df_.drop(['trip_duration'],axis=1, inplace=True)

In [ ]:
df_.head()

In [ ]:
test.head()

#### Definindo X e y

OBS: Alteração no valor do y somando 1 e tirando o log, para para ter valores negativos nas predições 

In [ ]:
X = df_.values[:,:-1]
y = np.log(df_['Duração'].values + 1)

In [ ]:
y.min()

### Divisão dos dados entre treino e testes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
y[0:5]

### Correlação dos dados

In [ ]:
corr = df_.corr()

sns.heatmap(corr)
plt.show()

In [ ]:
corr

In [ ]:
corr_matrix = corr
corr_matrix["Duração"].sort_values(ascending=False)

### Standard dos dados

In [ ]:
std = StandardScaler()
X_train_str = std.fit_transform(X_train)
X_test_str = std.transform(X_test)

### Modelos
    - LinearRegression
    - RandomForestRegressor
    - GradientBoostingRegressor
    - DecisionTreeRegressor

In [ ]:
models = []

models.append(('Linear', LinearRegression()))
models.append(('GBooting',GradientBoostingRegressor()))
models.append(('RFR', RandomForestRegressor(n_estimators=10)))
#models.append(('SVR', SVR()))
models.append(('DTR', DecisionTreeRegressor()))
#models.append(('KNN',KNeighborsRegressor))

 ### Treinando e calculando o RMSE e RMSLE nos modelos

In [ ]:
rmse_calc = []
rmsle = []

for nome,model in models:
    print(nome)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    rmse_calc.append((nome, np.sqrt(mean_squared_error(y_test,y_pred))))
    rmsle.append((nome,np.sqrt(mean_squared_log_error(y_test,y_pred))))

 ### Treinando e calculando o RMSE e RMSLE nos modelos com Standard

In [ ]:
rmse_str = []
rmsle_str = []

for nome,model in models:
    print(nome)
    model.fit(X_train_str,y_train)
    y_pred = model.predict(X_test_str)
    rmse_str.append((nome, np.sqrt(mean_squared_error(y_test,y_pred))))
    rmsle_str.append((nome,np.sqrt(mean_squared_log_error(y_test,y_pred))))

In [ ]:
rmsle_str

### Treinando e calculando o RMSE e RMSLE no XGBoosting

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [ ]:
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
watchlist = [(dtrain, 'train'), (dtest,'valid')]
xgb_pars = {'min_child_weight': 10, 'eta': 0.03, 'colsample_bytree': 0.3, 'max_depth': 10,
            'subsample': 0.8, 'lambda': 0.5, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

In [ ]:
model = xgb.train(xgb_pars, dtrain, 1000, watchlist,early_stopping_rounds=90,maximize=False, verbose_eval=100)

In [ ]:
y_pred = model.predict(dtest)

In [ ]:
y_pred

In [ ]:
rmse_xgb = np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
rmsle_xgb = np.sqrt(mean_squared_log_error(y_test,y_pred))

> ### RMSE e RMSLE de cada modelo

In [ ]:
rmse_calc

In [ ]:
rmsle

In [ ]:
nome_modelos = list(zip(*rmse_calc))[0]
index = np.arange(len(nome_modelos)) 

results_rmse = list(zip(*rmse_calc))[1]

bar_width = 0.55
opacity = 0.4
plt.figure(figsize=(12, 6))
plt.bar(index, results_rmse, bar_width, alpha=opacity, color='b', label='RMSE')
plt.xticks(index, nome_modelos) 
plt.xlabel('Modelos')
plt.ylabel('RSME')
plt.title('Comparação dos Modelos')
plt.show()

In [ ]:
nome_modelos = list(zip(*rmsle))[0]
index = np.arange(len(nome_modelos)) 

results_rmse = list(zip(*rmsle))[1]

bar_width = 0.55
opacity = 0.4
plt.figure(figsize=(12, 6))
plt.bar(index, results_rmse, bar_width, alpha=opacity, color='b', label='RMSLE')
plt.xticks(index, nome_modelos) 
plt.xlabel('Modelos')
plt.ylabel('RSMLE')
plt.title('Comparação dos Modelos')
plt.show()

In [ ]:
rmse_xgb

In [ ]:
model.best_score

In [ ]:
rmsle_xgb

## Exibindo a importancia das features para o modelo que obteve o melhor resultado

In [ ]:
df_.columns

In [ ]:
xgb.plot_importance(model, height=0.7);

## Submissão dos resultados

In [ ]:
dtest_k = xgb.DMatrix(test.values)

In [ ]:
pred = model.predict(dtest_k)

In [ ]:
pred.min()

In [ ]:
submission = pd.concat([id_test, pd.DataFrame(pred)], axis=1)
submission.columns = ['id','trip_duration']
submission.to_csv("submission.csv", index=False)